In [10]:
!nvidia-smi

Wed Dec 23 11:40:51 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.80.02    Driver Version: 450.80.02    CUDA Version: 11.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  GeForce RTX 208...  Off  | 00000000:3B:00.0 Off |                  N/A |
| 27%   28C    P8    20W / 250W |   1413MiB / 11016MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# 使用 ｐｉｐ 来确认下 

In [2]:
import pip 
for pkg in ['mxnet','mxnet-cu75','mxnet-cu80','mxnet-cu101']:
    pip.main(['show',pkg])

Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.
Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.
Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.
Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.


Name: mxnet-cu101
Version: 1.7.0
Summary: MXNet is an ultra-scalable deep learning framework. This version uses CUDA-10.1.
Home-page: https://github.com/apache/incubator-mxnet
Author: None
Author-email: None
License: Apache 2.0
Location: /home/dske/anaconda3/envs/gluon/lib/python3.6/site-packages
Requires: numpy, graphviz, requests
Required-by: 


# Context 
MXNET 使用 Context来指定使用哪个设备来存储和计算。默认会将数据开在主内成，然后利用CPU来计算，这个由mx.cpu()来表示。GPU则由mx.gpu()来表示。注意mx.cpu()表示所有的物理CPU和内成，意味着计算上会尽量使用多有的CPU核。但mx.gpu()只代表一块显卡和其对应的显卡内成。
如果有多块GPU，我们用mx.gpu(i)来表示第i快GPU （i 从0开始) .


In [4]:
import mxnet as mx
# [mx.cpu(),mx.gpu(),mx,gpu(1)]
[mx.cpu(),mx.gpu()]

[cpu(0), gpu(0)]

# NDArray 的GPU计算 
每个NDArray 都有一个context 属性来表示它存在哪个设备上，默认会是cpu。这是为什么前面每次我们打印NDArray的时候都会看到@cpu(0)这个标识 


In [5]:
from mxnet import nd
x=nd.array([1,2,3]) 
x.context 

cpu(0)

# GPU 上创建内存 
我们可以在创建的时候指定创建在哪个设备上（如果GPU不能用或者没有装MXnet GPU 版本，这里会有error 

In [7]:
a=nd.array([1,2,3],ctx=mx.gpu()) 
b=nd.zeros((3,2),ctx=mx.gpu())
c=nd.random.uniform(shape=(2,3),ctx=mx.gpu()) 
(a,b,c)

(
 [1. 2. 3.]
 <NDArray 3 @gpu(0)>,
 
 [[0. 0.]
  [0. 0.]
  [0. 0.]]
 <NDArray 3x2 @gpu(0)>,
 
 [[0.33810216 0.03127927 0.2290985 ]
  [0.7409259  0.51678467 0.48066086]]
 <NDArray 2x3 @gpu(0)>)

In [12]:
import sys 
try:
    nd.array([1,2,3],ctx=mx.gpu(1))
except mx.MXNetError as err:
    haha=err 
    sys.stderr.write(str(err))

Traceback (most recent call last):
  File "src/engine/./../common/cuda_utils.h", line 395
CUDA: Check failed: e == cudaSuccess || e == cudaErrorCudartUnloading: invalid device ordinal

我们可以通过copyto和 as_in_context来在设备上直接传输数据 。 


In [13]:
y=x.copyto(mx.gpu()) 
z=x.as_in_context(mx.gpu())
(y,z)

(
 [1. 2. 3.]
 <NDArray 3 @gpu(0)>,
 
 [1. 2. 3.]
 <NDArray 3 @gpu(0)>)

这两个函数的主要区别是，如果源和目标的context一致，as_in_context不复制，而copyto总是会新建内存：

In [14]:
yy=y.as_in_context(mx.gpu())
zz=z.copyto(mx.gpu())
(yy is y , zz is z )

(True, False)

# GPU 上的计算 
计算会在数据的context上执行。所以为了使用GPU,我们只需要事先将数据放在上面就行了。结果会自动保存在对应的设备设。

In [15]:
nd.exp(z+2)*y 


[ 20.085537 109.1963   445.2395  ]
<NDArray 3 @gpu(0)>

注意所有计算要求输入数据在同一个设备上，不一致的时候，系统不进行自动赋值。这个设计的目的是因为设备之间的数据交互通常
比较昂贵，我们希望用户确切置到数据放在哪里，而不是隐藏这个细节。下面代码尝试将CPU上和GPU 上的y做运输。 

## 默认会复制CPU的操作 
如果某个操作需要将NDArray里面的内容转出来，例如打印或变成numpy格式，如果需要的话，系统都会自动将数据copy到主内存 

In [17]:
print(y)
print(y.asnumpy())
print(y.sum().asscalar())


[1. 2. 3.]
<NDArray 3 @gpu(0)>
[1. 2. 3.]
6.0


## Gluon的GPU计算 
同NDArray类似，Gluon的大部分函数可以通过ctx指定设备。下面代码将模型参数初始化在GPU上:

In [18]:
from mxnet import gluon 
net=gluon.nn.Sequential()
net.add(gluon.nn.Dense(1))
net.initialize(ctx=mx.gpu())

#### 输入到GPU上的数据，会在GPU上计算结果　

In [19]:
data=nd.random.uniform(shape=[3,2],ctx=mx.gpu())
net(data)


[[0.00699626]
 [0.01652164]
 [0.00990379]]
<NDArray 3x1 @gpu(0)>

### 确认下权重 

In [20]:
net[0].weight.data()


[[0.0068339  0.01299825]]
<NDArray 1x2 @gpu(0)>

我们可以通过copyto和as_in_context来在设备直接传输数据。 

In [21]:
y=x.copyto(mx.gpu())
z=x.as_in_context(mx.gpu())
(z,y) 

(
 [1. 2. 3.]
 <NDArray 3 @gpu(0)>,
 
 [1. 2. 3.]
 <NDArray 3 @gpu(0)>)